# Peticiones con requests

Hasta ahora hemos visto como implementar una API con Python, pero ¿cómo interactuamos desde Python con una API? Pues generando peticiones HTTP directamente desde el código sin necesidad de un navegador. 

Para hacerlo tenemos a nuestra disposición un potente módulo externo llamado `requests`:

```bash
$ pip install requests
```

In [1]:
!pip install requests

## Primera petición GET

El módulo `requests` permite utilizar diferentes métodos como `get` y `post`, hagamos una prueba:

In [2]:
import requests

r = requests.get('https://www.wikipedia.es/')

print(type(r)) # tipo del dato

<class 'requests.models.Response'>


Podemos ver información de la respuesta analizando los atributos del objeto `Response`:

In [3]:
print(r.text) # contenido de la respuesta

<!DOCTYPE html>
<html class="client-nojs" lang="es" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Wikipedia, la enciclopedia libre</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"],"wgRequestId":"f450f076-21d0-4716-a8fe-e399ad50a90d","wgCSPNonce":false,"wgCanonicalNamespace":"Project","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":4,"wgPageName":"Wikipedia:Portada","wgTitle":"Portada","wgCurRevisionId":123425818,"wgRevisionId":123425818,"wgArticleId":2271189,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgPageContentLanguage":"es","wgPageContentModel":"wikitext","wgRelevantPageName":"Wikipedia:Portada","wgRelevantArticleId":2271189

In [4]:
print(r.status_code) # código de la respuesta

200


In [5]:
print(r.headers) # cabeceras de la respuesta

{'date': 'Mon, 15 Aug 2022 00:54:55 GMT', 'server': 'mw1393.eqiad.wmnet', 'x-content-type-options': 'nosniff', 'content-language': 'es', 'vary': 'Accept-Encoding,Cookie,Authorization', 'last-modified': 'Mon, 15 Aug 2022 00:54:47 GMT', 'content-type': 'text/html; charset=UTF-8', 'content-encoding': 'gzip', 'age': '34531', 'x-cache': 'cp6013 hit, cp6009 hit/1777', 'x-cache-status': 'hit-front', 'server-timing': 'cache;desc="hit-front", host;desc="cp6009"', 'strict-transport-security': 'max-age=106384710; includeSubDomains; preload', 'report-to': '{ "group": "wm_nel", "max_age": 86400, "endpoints": [{ "url": "https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&schema_uri=/w3c/reportingapi/network_error/1.0.0" }] }', 'nel': '{ "report_to": "wm_nel", "max_age": 86400, "failure_fraction": 0.05, "success_fraction": 0.0}', 'accept-ch': 'Sec-CH-UA-Arch,Sec-CH-UA-Bitness,Sec-CH-UA-Full-Version-List,Sec-CH-UA-Model,Sec-CH-UA-Platform-Version', 'permissions-policy'

In [6]:
print(r.headers['date'])             # fecha de la petición
print(r.headers['last-modified'])    # fecha de última modificación
print(r.headers['content-type'])     # tipo del contenido
print(r.headers['server'])           # servidor
print(r.headers['content-language']) # idioma

Mon, 15 Aug 2022 00:54:55 GMT
Mon, 15 Aug 2022 00:54:47 GMT
text/html; charset=UTF-8
mw1393.eqiad.wmnet
es


## Parámetros en las peticiones

La herramienta `httpbin` es un servicio de peticiones y respuestas HTTP gratuito que proporciona un conjunto de rutas para pruebas. Podemos hacer una petición a `https://httpbin.org/get` y pasarle algunos datos para que los retorne de vuelta.

Los parámetros GET se envían en el formato clave-valor, en Python podemos definirlos en un diccionario que generalmente recibe el nombre de `payload` (datos de carga):

In [7]:
url = 'https://httpbin.org/get'

payload = {
    'nombre': 'Hector Costa Guzman',
    'intereses':['Python', 'Videojuegos'],
    'edad': 33,
}

r = requests.get(url, params=payload)

print(r.text)

{
  "args": {
    "edad": "33", 
    "intereses": [
      "Python", 
      "Videojuegos"
    ], 
    "nombre": "Hector Costa Guzman"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.28.1", 
    "X-Amzn-Trace-Id": "Root=1-62fa2047-25c6b60e573044d22443e2e6"
  }, 
  "origin": "178.156.60.95", 
  "url": "https://httpbin.org/get?nombre=Hector+Costa+Guzman&intereses=Python&intereses=Videojuegos&edad=33"
}



La respuesta es una cadena en formato `json` que deberíamos *parsear* para extraer los datos, algo que veremos en la próxima lección.

## Peticiones POST

Como ya hemos comentado las peticiones POST sirven para enviar datos recopilados en un formulario. Podemos hacer la misma petición que antes utilizando `post` y cambiando la ruta a `http://httpbin.org/post`. 

Para enviar los datos tendremos que cambiar el argumento `params` por `data` para especificar datos de formularios:

In [8]:
url = 'https://httpbin.org/post'

payload = {
    'nombre': 'Hector Costa Guzman',
    'intereses':['Python', 'Videojuegos'],
    'edad': 33,
}

r = requests.post(url, data=payload)

print(r.text)

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "edad": "33", 
    "intereses": [
      "Python", 
      "Videojuegos"
    ], 
    "nombre": "Hector Costa Guzman"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "73", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.28.1", 
    "X-Amzn-Trace-Id": "Root=1-62fa2049-252c5b0579c8f6fc466bd6db"
  }, 
  "json": null, 
  "origin": "178.156.60.95", 
  "url": "https://httpbin.org/post"
}



En caso de que un servidor solo acepte `payload` en formato JSON hay que utilizar el argumento `json=payload` en lugar de `data=payload`:

In [9]:
url = 'https://httpbin.org/post'

payload = {
    'nombre': 'Hector Costa Guzman',
    'intereses':['Python', 'Videojuegos'],
    'edad': 33,
}

r = requests.post(url, json=payload)

print(r.text)

{
  "args": {}, 
  "data": "{\"nombre\": \"Hector Costa Guzman\", \"intereses\": [\"Python\", \"Videojuegos\"], \"edad\": 33}", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "85", 
    "Content-Type": "application/json", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.28.1", 
    "X-Amzn-Trace-Id": "Root=1-62fa204a-0b613fa32ae497425637b849"
  }, 
  "json": {
    "edad": 33, 
    "intereses": [
      "Python", 
      "Videojuegos"
    ], 
    "nombre": "Hector Costa Guzman"
  }, 
  "origin": "178.156.60.95", 
  "url": "https://httpbin.org/post"
}



## Errores en las peticiones

Es posible que en algunas ocasiones los servidores HTTP no se encuentren activos o las peticiones devuelvan un error.

Vamos a pedir a `httpbin` que simule una respuesta de error haciendo una petición GET a `https://httpbin.org/status/404`:

In [10]:
r = requests.get('https://httpbin.org/status/404')

Como veremos el código de respuesta es 404 (no encontrado):

In [11]:
print(r.status_code)

404


Este error podemos tratarlo en un bloque `try-except` de la siguiente forma:

In [12]:
import requests
from requests import HTTPError

try:
    # Hacemos la petición al sitio de pruebas
    r = requests.get("https://httpbin.org/status/404")
    # Pedimos que se invoque una excepcion HTTPError si hay algún fallo
    r.raise_for_status()
    # Si no hay un fallo mostramos algo de prueba
    print(r.status_code)
except HTTPError as ex:
    # Si se ha invocado una excepción HTTPError mostramos el resultado
    print(ex)

404 Client Error: NOT FOUND for url: https://httpbin.org/status/404


## Peticiones con timeout

Otra posibilidad es que en lugar de devolver un error el servidor simplemente no responda o tarde mucho tiempo.

Por defecto las peticiones no tienen un `timeout`, por lo que `requests` quedará en espera hasta que el servidor responda. Podemos simular esta situación estableciendo un tiempo de `delay` de 5 segundos en una petición a la ruta `https://httpbin.org/delay/5`:

In [13]:
r = requests.get('https://httpbin.org/delay/5')

In [14]:
print(r.status_code)

200


Sin embargo podemos indicarle un tiempo máximo para realizar la petición, por ejemplo si le indicamos 3 segundos fallará porque es menos que los 5 que tarda en responder:

In [15]:
r = requests.get('https://httpbin.org/delay/5', timeout=3)

ReadTimeout: HTTPSConnectionPool(host='httpbin.org', port=443): Read timed out. (read timeout=3)

Como vemos se devuelve un error `TimeoutError`, debemos envolver el bloque de código en un `try-except` y tratar este caso de forma específica:

In [16]:
from requests.exceptions import Timeout

try:
    requests.get('https://httpbin.org/delay/5', timeout=3)
except Timeout:
    print("ERROR: La petición ha tardado más de 3 segundos")

ERROR: La petición ha tardado más de 3 segundos


Las posibilidades del módulo `requests` son tantas como utilizar manualmente el navegador, os dejo la [documentación oficial](https://requests.readthedocs.io/en/latest/) para aprender más.